In [3]:
import requests
import json
import base64

# WordPress 사이트 설정
WP_URL = "https://prayer-church.co.kr"
WP_API = f"{WP_URL}/wp-json/wp/v2"
USERNAME = "prayerchurch"
APP_PASSWORD = "AziD L62i Ga4b 1efW ToAs hBMB"

# GA 및 GTM ID 설정
GA_ID = "G-QVYVVNPQGL"  # 실제 Google Analytics ID
GTM_ID = "GTM-K7GR4JW6"  # 실제 Google Tag Manager ID

# 인증 헤더 생성
credentials = f"{USERNAME}:{APP_PASSWORD}"
token = base64.b64encode(credentials.encode()).decode()
headers = {
    "Authorization": f"Basic {token}",
    "Content-Type": "application/json"
}

def create_custom_html_plugin():
    """
    Code Snippets 플러그인을 사용하여 GA 및 GTM 코드 추가
    """
    # GA4 스크립트
    ga_script = f"""
<!-- Google Analytics (GA4) -->
<script async src="https://www.googletagmanager.com/gtag/js?id={GA_ID}"></script>
<script>
  window.dataLayer = window.dataLayer || [];
  function gtag(){{dataLayer.push(arguments);}}
  gtag('js', new Date());
  gtag('config', '{GA_ID}');
</script>
"""
    
    # GTM 스크립트 (head에 들어갈 부분)
    gtm_head_script = f"""
<!-- Google Tag Manager -->
<script>(function(w,d,s,l,i){{w[l]=w[l]||[];w[l].push({{'gtm.start':
new Date().getTime(),event:'gtm.js'}});var f=d.getElementsByTagName(s)[0],
j=d.createElement(s),dl=l!='dataLayer'?'&l='+l:'';j.async=true;j.src=
'https://www.googletagmanager.com/gtm.js?id='+i+dl;f.parentNode.insertBefore(j,f);
}})(window,document,'script','dataLayer','{GTM_ID}');</script>
<!-- End Google Tag Manager -->
"""
    
    # GTM noscript 태그 (body 시작 부분에 추가)
    gtm_body_script = f"""
<!-- Google Tag Manager (noscript) -->
<noscript><iframe src="https://www.googletagmanager.com/ns.html?id={GTM_ID}"
height="0" width="0" style="display:none;visibility:hidden"></iframe></noscript>
<!-- End Google Tag Manager (noscript) -->
"""
    
    # Code Snippets 플러그인 API 엔드포인트 (통상적인 엔드포인트)
    snippets_endpoint = f"{WP_URL}/wp-json/code-snippets/v1/snippets"
    
    # 1. GA 및 GTM 헤드 스크립트용 스니펫 생성
    head_snippet_data = {
        "title": "Google Analytics and Tag Manager Header Script",
        "code": ga_script + gtm_head_script,
        "tags": ["analytics", "google"],
        "scope": "front-end",
        "active": True,
        "priority": 10,
        "location": "header" # 헤더에 삽입
    }
    
    # 2. GTM noscript용 스니펫 생성 (body 시작 부분에 삽입)
    body_snippet_data = {
        "title": "Google Tag Manager Body Script",
        "code": gtm_body_script,
        "tags": ["analytics", "google"],
        "scope": "front-end",
        "active": True,
        "priority": 1,
        "location": "content" # body 시작 부분에 삽입
    }
    
    try:
        # 헤더 스니펫 추가
        head_response = requests.post(
            snippets_endpoint, 
            headers=headers, 
            json=head_snippet_data
        )
        
        print(f"헤더 스니펫 응답 코드: {head_response.status_code}")
        if head_response.status_code in [200, 201]:
            print("GA 및 GTM 헤더 스크립트가 성공적으로 추가되었습니다.")
        else:
            print(f"헤더 스크립트 추가 실패: {head_response.text}")
        
        # Body 스니펫 추가
        body_response = requests.post(
            snippets_endpoint, 
            headers=headers, 
            json=body_snippet_data
        )
        
        print(f"Body 스니펫 응답 코드: {body_response.status_code}")
        if body_response.status_code in [200, 201]:
            print("GTM body 스크립트가 성공적으로 추가되었습니다.")
        else:
            print(f"Body 스크립트 추가 실패: {body_response.text}")
            
    except Exception as e:
        print(f"오류 발생: {e}")


def create_customizer_api_script():
    """
    테마 Customizer API를 통해 스크립트 추가 시도
    아스트라 테마의 경우 다른 엔드포인트를 사용할 수 있음
    """
    # 아스트라 테마 커스터마이저 API 엔드포인트
    astra_endpoint = f"{WP_URL}/wp-json/astra-sites/v1/customizer-settings"
    
    # GA 및 GTM 스크립트
    ga_script = f"""
<!-- Google Analytics (GA4) -->
<script async src="https://www.googletagmanager.com/gtag/js?id={GA_ID}"></script>
<script>
  window.dataLayer = window.dataLayer || [];
  function gtag(){{dataLayer.push(arguments);}}
  gtag('js', new Date());
  gtag('config', '{GA_ID}');
</script>
"""
    
    gtm_script = f"""
<!-- Google Tag Manager -->
<script>(function(w,d,s,l,i){{w[l]=w[l]||[];w[l].push({{'gtm.start':
new Date().getTime(),event:'gtm.js'}});var f=d.getElementsByTagName(s)[0],
j=d.createElement(s),dl=l!='dataLayer'?'&l='+l:'';j.async=true;j.src=
'https://www.googletagmanager.com/gtm.js?id='+i+dl;f.parentNode.insertBefore(j,f);
}})(window,document,'script','dataLayer','{GTM_ID}');</script>
<!-- End Google Tag Manager -->
"""
    
    # 아스트라 테마 설정 데이터
    settings_data = {
        "astra-settings": {
            "header-scripts": ga_script + gtm_script,
            "body-scripts": f"""
<!-- Google Tag Manager (noscript) -->
<noscript><iframe src="https://www.googletagmanager.com/ns.html?id={GTM_ID}"
height="0" width="0" style="display:none;visibility:hidden"></iframe></noscript>
<!-- End Google Tag Manager (noscript) -->
"""
        }
    }
    
    try:
        response = requests.post(
            astra_endpoint, 
            headers=headers, 
            json=settings_data
        )
        
        print(f"아스트라 테마 API 응답 코드: {response.status_code}")
        if response.status_code in [200, 201]:
            print("아스트라 테마 설정을 통해 GA 및 GTM 코드가 추가되었습니다.")
            return True
        else:
            print(f"아스트라 테마 설정 업데이트 실패: {response.text}")
            return False
            
    except Exception as e:
        print(f"오류 발생: {e}")
        return False


def create_options_api_script():
    """
    워드프레스 Options API를 사용하여 스크립트 추가
    """
    # 워드프레스 Options API 엔드포인트
    options_endpoint = f"{WP_URL}/wp-json/wp/v2/settings"
    
    # GA 및 GTM 스크립트 결합
    combined_head_script = f"""
<!-- Google Analytics (GA4) -->
<script async src="https://www.googletagmanager.com/gtag/js?id={GA_ID}"></script>
<script>
  window.dataLayer = window.dataLayer || [];
  function gtag(){{dataLayer.push(arguments);}}
  gtag('js', new Date());
  gtag('config', '{GA_ID}');
</script>

<!-- Google Tag Manager -->
<script>(function(w,d,s,l,i){{w[l]=w[l]||[];w[l].push({{'gtm.start':
new Date().getTime(),event:'gtm.js'}});var f=d.getElementsByTagName(s)[0],
j=d.createElement(s),dl=l!='dataLayer'?'&l='+l:'';j.async=true;j.src=
'https://www.googletagmanager.com/gtm.js?id='+i+dl;f.parentNode.insertBefore(j,f);
}})(window,document,'script','dataLayer','{GTM_ID}');</script>
<!-- End Google Tag Manager -->
"""
    
    # Settings API 데이터
    settings_data = {
        "headcode": combined_head_script,
        "bodycode": f"""
<!-- Google Tag Manager (noscript) -->
<noscript><iframe src="https://www.googletagmanager.com/ns.html?id={GTM_ID}"
height="0" width="0" style="display:none;visibility:hidden"></iframe></noscript>
<!-- End Google Tag Manager (noscript) -->
"""
    }
    
    try:
        response = requests.post(
            options_endpoint, 
            headers=headers, 
            json=settings_data
        )
        
        print(f"설정 API 응답 코드: {response.status_code}")
        if response.status_code in [200, 201]:
            print("워드프레스 설정을 통해 GA 및 GTM 코드가 추가되었습니다.")
            return True
        else:
            print(f"설정 업데이트 실패: {response.text}")
            return False
            
    except Exception as e:
        print(f"오류 발생: {e}")
        return False


def create_custom_post_for_script():
    """
    사용자 정의 포스트 타입을 사용하여 스크립트를 저장하고 플러그인이 이를 사용하도록 설정
    """
    # 사용자 정의 포스트 타입 엔드포인트 (예: 'code_snippet')
    post_endpoint = f"{WP_API}/code_snippet"
    
    # GA 스크립트
    ga_post_data = {
        "title": "Google Analytics Script",
        "content": f"""
<!-- Google Analytics (GA4) -->
<script async src="https://www.googletagmanager.com/gtag/js?id={GA_ID}"></script>
<script>
  window.dataLayer = window.dataLayer || [];
  function gtag(){{dataLayer.push(arguments);}}
  gtag('js', new Date());
  gtag('config', '{GA_ID}');
</script>
""",
        "status": "publish",
        "meta": {
            "location": "header",
            "active": True
        }
    }
    
    # GTM 스크립트
    gtm_post_data = {
        "title": "Google Tag Manager Script",
        "content": f"""
<!-- Google Tag Manager -->
<script>(function(w,d,s,l,i){{w[l]=w[l]||[];w[l].push({{'gtm.start':
new Date().getTime(),event:'gtm.js'}});var f=d.getElementsByTagName(s)[0],
j=d.createElement(s),dl=l!='dataLayer'?'&l='+l:'';j.async=true;j.src=
'https://www.googletagmanager.com/gtm.js?id='+i+dl;f.parentNode.insertBefore(j,f);
}})(window,document,'script','dataLayer','{GTM_ID}');</script>
<!-- End Google Tag Manager -->
""",
        "status": "publish",
        "meta": {
            "location": "header",
            "active": True
        }
    }
    
    # GTM noscript 태그
    gtm_body_post_data = {
        "title": "Google Tag Manager Body Script",
        "content": f"""
<!-- Google Tag Manager (noscript) -->
<noscript><iframe src="https://www.googletagmanager.com/ns.html?id={GTM_ID}"
height="0" width="0" style="display:none;visibility:hidden"></iframe></noscript>
<!-- End Google Tag Manager (noscript) -->
""",
        "status": "publish",
        "meta": {
            "location": "body_open",
            "active": True
        }
    }
    
    try:
        # GA 코드 추가
        ga_response = requests.post(
            post_endpoint, 
            headers=headers, 
            json=ga_post_data
        )
        
        print(f"GA 코드 응답: {ga_response.status_code}")
        if ga_response.status_code in [200, 201]:
            print("GA 코드가 성공적으로 추가되었습니다.")
        else:
            print(f"GA 코드 추가 실패: {ga_response.text}")
        
        # GTM 코드 추가
        gtm_response = requests.post(
            post_endpoint, 
            headers=headers, 
            json=gtm_post_data
        )
        
        print(f"GTM 코드 응답: {gtm_response.status_code}")
        if gtm_response.status_code in [200, 201]:
            print("GTM 코드가 성공적으로 추가되었습니다.")
        else:
            print(f"GTM 코드 추가 실패: {gtm_response.text}")
        
        # GTM body 코드 추가
        gtm_body_response = requests.post(
            post_endpoint, 
            headers=headers, 
            json=gtm_body_post_data
        )
        
        print(f"GTM body 코드 응답: {gtm_body_response.status_code}")
        if gtm_body_response.status_code in [200, 201]:
            print("GTM body 코드가 성공적으로 추가되었습니다.")
        else:
            print(f"GTM body 코드 추가 실패: {gtm_body_response.text}")
            
    except Exception as e:
        print(f"오류 발생: {e}")


def test_insert_headers_plugin_api():
    """
    Insert Headers and Footers 플러그인 API 테스트
    """
    # 플러그인 API 엔드포인트
    plugin_endpoint = f"{WP_URL}/wp-json/wp-headers-and-footers/v2/settings"
    
    # GA 및 GTM 스크립트
    header_code = f"""
<!-- Google Analytics (GA4) -->
<script async src="https://www.googletagmanager.com/gtag/js?id={GA_ID}"></script>
<script>
  window.dataLayer = window.dataLayer || [];
  function gtag(){{dataLayer.push(arguments);}}
  gtag('js', new Date());
  gtag('config', '{GA_ID}');
</script>

<!-- Google Tag Manager -->
<script>(function(w,d,s,l,i){{w[l]=w[l]||[];w[l].push({{'gtm.start':
new Date().getTime(),event:'gtm.js'}});var f=d.getElementsByTagName(s)[0],
j=d.createElement(s),dl=l!='dataLayer'?'&l='+l:'';j.async=true;j.src=
'https://www.googletagmanager.com/gtm.js?id='+i+dl;f.parentNode.insertBefore(j,f);
}})(window,document,'script','dataLayer','{GTM_ID}');</script>
<!-- End Google Tag Manager -->
"""
    
    body_code = f"""
<!-- Google Tag Manager (noscript) -->
<noscript><iframe src="https://www.googletagmanager.com/ns.html?id={GTM_ID}"
height="0" width="0" style="display:none;visibility:hidden"></iframe></noscript>
<!-- End Google Tag Manager (noscript) -->
"""
    
    # 플러그인 설정 데이터
    settings_data = {
        "wp_headers_and_footers_settings_header": header_code,
        "wp_headers_and_footers_settings_body": body_code,
        "wp_headers_and_footers_settings_footer": ""
    }
    
    try:
        response = requests.post(
            plugin_endpoint, 
            headers=headers, 
            json=settings_data
        )
        
        print(f"Insert Headers and Footers 플러그인 응답 코드: {response.status_code}")
        if response.status_code in [200, 201]:
            print("Insert Headers and Footers 플러그인을 통해 GA 및 GTM 코드가 추가되었습니다.")
            return True
        else:
            print(f"플러그인 설정 업데이트 실패: {response.text}")
            return False
            
    except Exception as e:
        print(f"오류 발생: {e}")
        return False


def discover_available_endpoints():
    """
    사용 가능한 API 엔드포인트를 검색
    """
    try:
        # 워드프레스 REST API 루트 엔드포인트에서 사용 가능한 엔드포인트 정보를 가져옴
        root_endpoint = f"{WP_URL}/wp-json"
        response = requests.get(root_endpoint, headers=headers)
        
        if response.status_code == 200:
            routes = response.json().get("routes", {})
            print("사용 가능한 API 엔드포인트:")
            for route in routes:
                print(f"  - {route}")
                
            # 특정 키워드로 엔드포인트 필터링
            keywords = ["header", "script", "customiz", "setting", "option", "snippet", "code"]
            print("\n관련 가능성이 있는 엔드포인트:")
            for route in routes:
                if any(keyword in route.lower() for keyword in keywords):
                    print(f"  - {route}")
        else:
            print(f"API 루트 엔드포인트 액세스 실패: {response.status_code}")
            print(response.text)
    except Exception as e:
        print(f"API 엔드포인트 검색 중 오류 발생: {e}")


def main():
    """
    메인 함수 - 모든 방법을 순차적으로 시도
    """
    print("WordPress 사이트에 GA 및 GTM 설치 스크립트 실행 중...")
    print(f"사용할 Google Analytics ID: {GA_ID}")
    print(f"사용할 Google Tag Manager ID: {GTM_ID}")
    
    # 1. 먼저 사용 가능한 엔드포인트 검색
    print("\n1. 사용 가능한 API 엔드포인트 검색 중...")
    discover_available_endpoints()
    
    # 2. Code Snippets 플러그인 API 시도
    print("\n2. Code Snippets 플러그인 API 시도 중...")
    create_custom_html_plugin()
    
    # 3. 아스트라 테마 API 시도
    print("\n3. 아스트라 테마 API 시도 중...")
    create_customizer_api_script()
    
    # 4. 워드프레스 Options API 시도
    print("\n4. 워드프레스 Options API 시도 중...")
    create_options_api_script()
    
    # 5. Insert Headers and Footers 플러그인 API 시도
    print("\n5. Insert Headers and Footers 플러그인 API 시도 중...")
    test_insert_headers_plugin_api()
    
    # 6. 사용자 정의 포스트 타입 시도
    print("\n6. 사용자 정의 포스트 타입 시도 중...")
    create_custom_post_for_script()
    
    print("\n스크립트 실행 완료. 각 방법의 결과를 확인하고 사이트를 테스트하세요.")
    print("여러 방법이 시도되었으므로, 중복 코드가 삽입되었을 수 있습니다. 이 경우 불필요한 코드를 제거하세요.")


if __name__ == "__main__":
    main()

WordPress 사이트에 GA 및 GTM 설치 스크립트 실행 중...
사용할 Google Analytics ID: G-QVYVVNPQGL
사용할 Google Tag Manager ID: GTM-K7GR4JW6

1. 사용 가능한 API 엔드포인트 검색 중...
사용 가능한 API 엔드포인트:
  - /
  - /batch/v1
  - /give-api/v2
  - /give-api/v2/admin/forms/migrate/(?P<id>\d+)
  - /give-api/v2/admin/forms/transfer
  - /oembed/1.0
  - /oembed/1.0/embed
  - /oembed/1.0/proxy
  - /file-manager-advanced/v1
  - /file-manager-advanced/v1/hide-banner
  - /file-manager-advanced/v1/minimize-maximize-banner
  - /litespeed/v1
  - /litespeed/v1/toggle_crawler_state
  - /litespeed/v1/tool/check_ip
  - /litespeed/v1/ip_validate
  - /litespeed/v1/token
  - /litespeed/v1/ping
  - /litespeed/v1/apikey
  - /litespeed/v1/cdn_status
  - /litespeed/v1/notify_img
  - /litespeed/v1/notify_ucss
  - /litespeed/v1/notify_vpi
  - /litespeed/v1/err_domains
  - /litespeed/v1/check_img
  - /presto-player/v1
  - /presto-player/v1/preset
  - /presto-player/v1/preset/(?P<id>\d+)
  - /presto-player/v1/audio-preset
  - /presto-player/v1/audio-pre

헤더 스니펫 응답 코드: 404
헤더 스크립트 추가 실패: {"code":"rest_no_route","message":"URL\uacfc \uc694\uccad\ud55c \uba54\uc18c\ub4dc\uc5d0 \uc77c\uce58\ud558\ub294 \ub77c\uc6b0\ud2b8\ub97c \ucc3e\uc744 \uc218 \uc5c6\uc2b5\ub2c8\ub2e4.","data":{"status":404}}
Body 스니펫 응답 코드: 404
Body 스크립트 추가 실패: {"code":"rest_no_route","message":"URL\uacfc \uc694\uccad\ud55c \uba54\uc18c\ub4dc\uc5d0 \uc77c\uce58\ud558\ub294 \ub77c\uc6b0\ud2b8\ub97c \ucc3e\uc744 \uc218 \uc5c6\uc2b5\ub2c8\ub2e4.","data":{"status":404}}

3. 아스트라 테마 API 시도 중...
아스트라 테마 API 응답 코드: 404
아스트라 테마 설정 업데이트 실패: {"code":"rest_no_route","message":"URL\uacfc \uc694\uccad\ud55c \uba54\uc18c\ub4dc\uc5d0 \uc77c\uce58\ud558\ub294 \ub77c\uc6b0\ud2b8\ub97c \ucc3e\uc744 \uc218 \uc5c6\uc2b5\ub2c8\ub2e4.","data":{"status":404}}

4. 워드프레스 Options API 시도 중...
설정 API 응답 코드: 200
워드프레스 설정을 통해 GA 및 GTM 코드가 추가되었습니다.

5. Insert Headers and Footers 플러그인 API 시도 중...
Insert Headers and Footers 플러그인 응답 코드: 404
플러그인 설정 업데이트 실패: {"code":"rest_no_route","message":"URL\uacfc 